# Tâche #2 : Classification d'incidents avec un réseau  récurrent et des *embeddings* Spacy

Cette tâche est similaire à la précédente et vous réutilisez les mêmes fichiers d’entraînement, de validation et de test. Cependant, vous devez utiliser des réseaux récurrents pour classifier les textes. Plus particulièrement, vous devez entraîner un réseau de neurones LSTM pour encoder les textes et une couche linéaire pour faire la classification des textes. 

Les consignes pour cette tâche sont: 
- 	Nom du notebook : rnn.ipynb
- 	Tokenisation : Utilisation de Spacy. 
- 	Plongements de mots : Ceux de Spacy. 
- 	Normalisation : Aucune normalisation. 
- 	Structure du réseau : Un réseau LSTM avec 1 seule couche pour l’encodage de textes. Je vous laisse déterminer la taille de cette couche (à expliquer). 
- 	Analyse : Comparer les résultats obtenus avec un réseau unidirectionnel et un réseau bidirectionnel. Si vous éprouvez des difficultés à entraîner les 2 réseaux dans un même notebook, faites une copie et nommez le 2e fichier rnn-bidirectionnel.ipynb.
- 	Expliquez comment les modèles sont utilisés pour faire la classification d’un texte. 
- 	Présentez clairement vos résultats et faites-en l’analyse. 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [170]:
import json

def load(filename: str):
    dataset = {}
    with open(filename, "r") as fp:
        incident_list = json.load(fp)
    texts, labels = zip(*[(incident.get("text"), int(incident.get("label"))) for incident in incident_list])
    dataset["text"] = list(texts)
    dataset["label"] = list(labels)

    return dataset

In [171]:
dataset = load('./data/incidents_train.json')

#### 1.1 Conversion en Identifiants

In [172]:

labels = list(set(dataset["label"]))
id2label = {int(label): label for label in labels}
label2id = {label: int(label) for label in labels}
nb_labels = len(labels)

## 2. Gestion de plongements de mots (*embeddings*)

In [173]:
import spacy

model = spacy.load('en_core_web_md')

embedding_size = model.meta['vectors']['width']

In [174]:
from enum import Enum
import numpy as np

class Tokens(Enum):
    PADDING = "<PAD>"
    UNKNOWN = "<UNK>"

zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = Tokens.PADDING
id2word[1] = Tokens.UNKNOWN

word2id = {}
word2id[Tokens.PADDING] = 0
word2id[Tokens.UNKNOWN] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding


In [175]:
def map_ids():
    word_index = 2
    vocabulary = word2id.keys()

    for sentence in dataset["text"]:
        for word in model(sentence):
            if word.text not in vocabulary:
                word2id[word.text] = word_index
                id2word[word_index] = word.text
                id2embedding[word_index] = word.vector
                word_index += 1

map_ids()

In [176]:
import torch

from torch import LongTensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

class QuestionDataset(Dataset):
    def __init__(self, data , targets, word_to_id, spacy_model):
        self.data = data
        self.sequences = [None for _ in range(len(data))]
        self.targets = targets
        self.word2id = word_to_id
        self.tokenizer = spacy_model

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.sequences[index] is None:
            self.sequences[index] = self.tokenize(self.data[index])
        
        return LongTensor(self.sequences[index]), LongTensor([self.targets[index]]).squeeze(0)

    def tokenize(self, sentence):
        tokens = [word.text for word in self.tokenizer(sentence)]
        
        return [self.word2id.get(token, 1) for token in tokens] 


train_dataset = QuestionDataset(dataset['text'], dataset['label'], word2id, model)

dataset = load('./data/incidents_dev.json')

validation_dataset = QuestionDataset(dataset['text'], dataset['label'], word2id, model)


## 3. Création de modèle(s)

In [177]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Rnn(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes, bidirectional=False):
        super(Rnn, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        self.embedding_size = embeddings.size()[1]
        self.lstm = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True, bidirectional=bidirectional)
        self.classification_layer = nn.Linear(hidden_state_size * 2 if bidirectional else hidden_state_size, nb_classes)
        self.hidden_state = None
        self.context = None
        self.bidirectional = bidirectional

    def forward(self, x, x_lengths):
        x = self.embedding_layer(x)
        packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        _, (self.hidden_state, self.context) = self.lstm(packed_batch)

        if self.lstm.bidirectional:
            x = torch.cat((self.hidden_state[0], self.hidden_state[1]), 1)
            x = self.classification_layer(x)
        else:
            x = self.hidden_state.squeeze()
            x = self.classification_layer(x)

        return x

In [178]:
def pad_batch(batch):
    x = [x for x,y in batch]
    x_true_length = [len(x) for x,y in batch]
    y = torch.stack([y for x,y in batch], dim=0)
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)

validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)


In [179]:
vocab_size = len(id2embedding)

embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)

for token_id, embedding in id2embedding.items():
    embedding_layer[token_id,:] = embedding
    
embedding_layer = torch.from_numpy(embedding_layer)

In [180]:
from poutyne import set_seeds

set_seeds(42)

hidden_size = 100 

network = Rnn(embedding_layer, hidden_size, nb_labels, False)

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

In [181]:
# def train_model_bundle(network, name, epochs=5):
    # """
    # This function creates a Poutyne ModelBundle, trains the input module
    # on the train loader and then tests its performance on the test loader.
    # All training and testing statistics are saved, as well as best model
    # checkpoints.
# 
    # Args:
        # network (torch.nn.Module): The neural network to train.
        # working_directory (str): The directory where to output files to save.
        # epochs (int): The number of epochs. (Default: 5)
    # """
    # print(network)
# 
    # optimizer = optim.SGD(network.parameters(), lr=learning_rate)
# 
    # save_path = os.path.join('saves', name)
# 
    # model_bundle = ModelBundle.from_network(
        # save_path,
        # network,
        # device=device,
        # optimizer=optimizer,
        # task='classif',
    # )
# 
    # model_bundle.train(train_loader, valid_loader, epochs=epochs)
# 
    # model_bundle.test(test_loader)

## 5. Entraînement de modèle(s)

In [183]:
from poutyne.framework import Experiment
import torch.optim as optim
from torch.nn.functional import cross_entropy

learning_rate = 0.1

optimizer = optim.SGD(network.parameters(), learning_rate)

model_directory = 'model/lstm_classification_bidirectional/' if network.bidirectional else 'model/lstm_classification/'

experiment = Experiment(model_directory, network, optimizer=optimizer, task="classification", loss_function=cross_entropy)


Loading weights from model/lstm_classification/checkpoint.ckpt and starting at epoch 26.
Loading optimizer state from model/lstm_classification/checkpoint.optim and starting at epoch 26.
Loading random states from model/lstm_classification/checkpoint.randomstate and starting at epoch 26.
Restoring data from model/lstm_classification/checkpoint_epoch_21.ckpt


In [ ]:

logging = experiment.train(train_dataloader, validation_dataloader, epochs=3, disable_tensorboard=True)

## 6. Évaluation et analyse de résultats

In [ ]:

dataset = load('./data/incidents_test.json')

test_dataset = QuestionDataset(dataset['text'], dataset['label'], word2id, model)

test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)

In [242]:
experiment.test(test_dataloader)

Found best checkpoint at epoch: 21
lr: 0.1, loss: 0.125539, acc: 98.3051, fscore_macro: 0.924582, val_loss: 0.10593, val_acc: 99.2467, val_fscore_macro: 0.930552
Loading checkpoint model/lstm_classification/checkpoint_epoch_21.ckpt
Running test
Test steps: 34 2.63s test_loss: 2.026285 test_acc: 49.529190 test_fscore_macro: 0.279179          


{'time': 2.6329579999437556,
 'test_loss': 2.026285367412756,
 'test_acc': 49.52919020715631,
 'test_fscore_macro': 0.2791789174079895}